### Data Transformation

In [1]:
import os

%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [5]:
from src.text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.text_summarizer.utils.common import create_directories, read_yaml


In [6]:
class ConfigurationManager:
    def __init__(self, config_path: str = CONFIG_FILE_PATH, params_filepath: str = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_path)
        self.paramss = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )

        return data_transformation_config


In [7]:
import os

from datasets import load_from_disk
from transformers import AutoTokenizer

from src.text_summarizer.logging import logger


[2025-06-25 01:21:31,324: INFO: config: PyTorch version 2.7.1 available.]


/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Transformation Component


In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch: dict) -> dict:
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length=128, truncation=True)

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"],
        }

    def convert(self) -> None:
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(Path(self.config.root_dir) / "samsum_dataset")

In [9]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-06-25 01:21:32,966: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 01:21:32,967: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 01:21:32,967: INFO: common: created directory at: artifacts]
[2025-06-25 01:21:32,968: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 183276.75 examples/s]
